In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
import os
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

In [3]:
# Load packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from collections import Counter, defaultdict
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.linear_model import LassoCV, LogisticRegression, LogisticRegressionCV, ElasticNetCV, ElasticNet
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import LeaveOneOut, KFold, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import statistics
import xgboost as xgb

In [ ]:
# Load dataset
species_data = pd.read_excel('species_abundance_merged_2024-08-13.xlsx')
pathway_data = pd.read_excel('pathway_abundance_merged_2024-08-21.xlsx')

# List of columns to merge on
merge_columns = ['SampleID', 'Subject', 'Subject_number', 'timepoint_numeric', 'Diagnosis', 'CD_onset', 'Relative_timepoint', 'Country']

# Perform the merge
combined_data = pd.merge(species_data, pathway_data, on=merge_columns, how='inner')
combined_data

In [5]:
# Exclude subjects with a CD onset of 12 months
excluded_subjects = [23, 31]
combined_data = combined_data[~combined_data['Subject_number'].isin(excluded_subjects)]

In [ ]:
# Function to apply abundance and prevalence thresholds
def filter_features(data, abundance_threshold = 0.001, prevalence_threshold = 0.1):
    initial_features_count = data.shape[1] - 8
    sample_count = data.shape[0]
    
    # Calculate prevalence threshold
    min_prevalent_samples = int(prevalence_threshold * sample_count)

    # Filter features based on the thresholds
    features_columns = data.columns.difference(['SampleID', 'Subject', 'Subject_number', 'timepoint_numeric', 'Diagnosis', 'CD_onset', 'Relative_timepoint', 'Country'])
    features_data = data[features_columns]
    
    # Calculate the abundance and prevalence for each feature
    features_above_threshold = (features_data >= abundance_threshold).sum(axis=0) >= min_prevalent_samples
    filtered_features = features_columns[features_above_threshold]
    
    # Filter the data to keep only the selected species
    filtered_data = data[['SampleID', 'Subject', 'Subject_number', 'timepoint_numeric', 'Diagnosis', 'CD_onset', 'Relative_timepoint', 'Country'] + filtered_features.tolist()]

    final_features_count = len(filtered_features)
    print(f"Initial number of features: {initial_features_count}")
    print(f"Number of features after filtering: {final_features_count}")
    
    return filtered_data

# Filter the data
combined_data_filtered = filter_features(combined_data)

In [ ]:
from sklearn.metrics import f1_score

# Define selected features for each time point
features_timepoint_12 = [
    'Coprococcus_comes', 'Bifidobacterium_dentium', 'Enterocloster_clostridioformis', 'Veillonella_ratti', 'Hungatella_hathewayi', 
    'Faecalimonas_umbilicata', 'Escherichia_coli', 'GGB9469_SGB14862', 'Tyzzerella_nexilis', 'Streptococcus_thermophilus', 
    'Alistipes_onderdonkii', 'Eggerthella_lenta', 'Roseburia_faecis', 'Bifidobacterium_pseudocatenulatum', 
    'Bacteroides_stercoris', 'Alistipes_putredinis', 'GGB51647_SGB4348', 'Lachnospira_pectinoschiza', 'Ruminococcus_lactaris', 
    'Faecalibacterium_sp_HTFF', 'Bifidobacterium_breve', 'Faecalibacterium_prausnitzii', 'Megasphaera_micronuciformis', 
    'GGB9480_SGB14874', 'Enterococcus_faecalis', 
    'PWY-5464: superpathway of cytosolic glycolysis (plants), pyruvate dehydrogenase and TCA cycle', 
    'PWY-6143: CMP-pseudaminate biosynthesis', 'PWY-5861: superpathway of demethylmenaquinol-8 biosynthesis I', 
    'PWY-5838: superpathway of menaquinol-8 biosynthesis I', 'PWY-5723: Rubisco shunt', 
    'POLYAMINSYN3-PWY: superpathway of polyamine biosynthesis II', 'PWY-5920: superpathway of heme b biosynthesis from glycine', 
    'GLYOXYLATE-BYPASS: glyoxylate cycle', 'P23-PWY: reductive TCA cycle I', 
    'PWY-5189: tetrapyrrole biosynthesis II (from glycine)', 'P105-PWY: TCA cycle IV (2-oxoglutarate decarboxylase)', 
    'GALACTITOLCAT-PWY: galactitol degradation', 'GLUCARDEG-PWY: D-glucarate degradation I', 
    'METHGLYUT-PWY: superpathway of methylglyoxal degradation', 
    'PWY-561: superpathway of glyoxylate cycle and fatty acid degradation', 
    'PWY-5860: superpathway of demethylmenaquinol-6 biosynthesis I', 
    'ASPASN-PWY: superpathway of L-aspartate and L-asparagine biosynthesis', 'PWY-5656: mannosylglycerate biosynthesis I', 
    'PWY-5840: superpathway of menaquinol-7 biosynthesis', 'HEXITOLDEGSUPER-PWY: superpathway of hexitol degradation (bacteria)', 
    'BIOTIN-BIOSYNTHESIS-PWY: biotin biosynthesis I', 'PWY-5345: superpathway of L-methionine biosynthesis (by sulfhydrylation)', 
    'PWY-5705: allantoin degradation to glyoxylate III', 'POLYISOPRENSYN-PWY: polyisoprenoid biosynthesis (E. coli)', 
    'PWY-5136: fatty acid &beta;-oxidation II (plant peroxisome)', 'NAGLIPASYN-PWY: lipid IVA biosynthesis (E. coli)', 
    'PWY-5896: superpathway of menaquinol-10 biosynthesis', 'PWY-5850: superpathway of menaquinol-6 biosynthesis', 
    'PWY-5686: UMP biosynthesis I', 'GLUCARGALACTSUPER-PWY: superpathway of D-glucarate and D-galactarate degradation', 
    'GALACTARDEG-PWY: D-galactarate degradation I', 'PWY-5897: superpathway of menaquinol-11 biosynthesis', 
    'PWY-5898: superpathway of menaquinol-12 biosynthesis', 'PWY-5899: superpathway of menaquinol-13 biosynthesis', 
    'GLYCOCAT-PWY: glycogen degradation I', 'PWY-5981: CDP-diacylglycerol biosynthesis III', 
    'PWY-5505: L-glutamate and L-glutamine biosynthesis', 'P124-PWY: Bifidobacterium shunt', 
    'ARGDEG-PWY: superpathway of L-arginine, putrescine, and 4-aminobutanoate degradation', 
    'PWY-5004: superpathway of L-citrulline metabolism', 'PWY-5862: superpathway of demethylmenaquinol-9 biosynthesis', 
    'PWY-5910: superpathway of geranylgeranyldiphosphate biosynthesis I (via mevalonate)', 'P221-PWY: octane oxidation', 
    'PWY-6292: superpathway of L-cysteine biosynthesis (mammalian)', 'PWY-5675: nitrate reduction V (assimilatory)', 
    'PWY-5845: superpathway of menaquinol-9 biosynthesis', 
    'HCAMHPDEG-PWY: 3-phenylpropanoate and 3-(3-hydroxyphenyl)propanoate degradation to 2-hydroxypentadienoate'
]

features_timepoint_15 = [
    'Veillonella_ratti', 'Clostridium_sp_AT4', 'Bifidobacterium_pseudocatenulatum', 'Lacticaseibacillus_rhamnosus', 'Faecalibacterium_sp_HTFF', 
    'Phocaeicola_dorei', 'Enterococcus_faecalis', 'Ruminococcus_bromii', 'Dorea_longicatena', 'Lachnospira_pectinoschiza', 'Lacrimispora_amygdalina', 
    'Eggerthella_lenta', 'Clostridium_SGB6179', 'Ruminococcus_gnavus', 'Sellimonas_intestinalis', 'Akkermansia_muciniphila', 'GGB9480_SGB14874', 
    'Escherichia_coli', 'PWY-6143: CMP-pseudaminate biosynthesis', 'PWY-5920: superpathway of heme b biosynthesis from glycine', 
    'PWY-5514: UDP-N-acetyl-D-galactosamine biosynthesis II', 'GLUCARDEG-PWY: D-glucarate degradation I', 
    'PWY-5910: superpathway of geranylgeranyldiphosphate biosynthesis I (via mevalonate)', 
    'FAO-PWY: fatty acid &beta;-oxidation I (generic)', 'HEXITOLDEGSUPER-PWY: superpathway of hexitol degradation (bacteria)', 
    'PWY-5675: nitrate reduction V (assimilatory)', 'PWY-5850: superpathway of menaquinol-6 biosynthesis', 
    'PWY-5896: superpathway of menaquinol-10 biosynthesis', 
    'HCAMHPDEG-PWY: 3-phenylpropanoate and 3-(3-hydroxyphenyl)propanoate degradation to 2-hydroxypentadienoate', 
    'METHGLYUT-PWY: superpathway of methylglyoxal degradation', 'HEME-BIOSYNTHESIS-II: heme b biosynthesis I (aerobic)', 
    'FUC-RHAMCAT-PWY: superpathway of fucose and rhamnose degradation', 'KETOGLUCONMET-PWY: ketogluconate metabolism', 
    'P621-PWY: nylon-6 oligomer degradation', 'FUCCAT-PWY: fucose degradation', 
    'GLYCOLYSIS-E-D: superpathway of glycolysis and the Entner-Doudoroff pathway', 'HISDEG-PWY: L-histidine degradation I', 
    'PWY-5367: petroselinate biosynthesis', 'POLYISOPRENSYN-PWY: polyisoprenoid biosynthesis (E. coli)', 
    'PWY-5189: tetrapyrrole biosynthesis II (from glycine)', 'PWY-1861: formaldehyde assimilation II (assimilatory RuMP Cycle)'
]

# Define a function to evaluate combinations of feature selectors and models
def evaluate_models(data, max_timepoint):
    results = {}

    # Define a function to process each time point independently
    def process_time_point(time_point):
        print(f"Processing time point: {time_point}")

        # Select features based on the time point
        if time_point == 12:
            selected_features = features_timepoint_12
        elif time_point == 15:
            selected_features = features_timepoint_15
        else:
            print(f"No predefined features for time point {time_point}.")
            return time_point, None

        # Filter data for the current time point and extract selected features
        current_data = data[data['timepoint_numeric'] == time_point]
        X = current_data[selected_features] # Feature matrix
        y = current_data['Diagnosis'] # Labels

        # Ensure there are enough samples to perform LOOCV
        if len(y) < 2:
            print(f"Skipping time point {time_point} due to insufficient samples.")
            return time_point, None

        # Initialize Leave-One-Out cross-validation
        loo = LeaveOneOut()
        best_overall_score = 0
        best_overall_setup = {}

        # Define a function to process each feature selector and model combination
        def process_combination(feature_selector_name, ml_model_name):
            all_selected_features = []
            all_importances = []

            # Loop through the training/test splits generated by LOOCV
            for train_index, test_index in loo.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # Perform feature selection based on the specified selector
                if feature_selector_name == 'LASSO':
                    feature_selector = LassoCV(cv=5, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30)).fit(X_train, y_train)
                else:
                    feature_selector = ElasticNetCV(cv=5, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7).fit(X_train, y_train)

                # Select features with non-zero coefficients
                selected_features = X_train.columns[feature_selector.coef_ != 0]
                selected_features = selected_features[:min(len(selected_features), int(0.8 * len(y_train)))]

                # If no features are selected, skip this iteration
                if len(selected_features) == 0:
                    continue

                # Select the relevant features from the training set
                X_train_selected = X_train[selected_features]

                # Perform logistic regression for ranking the selected features based on importance
                logistic = LogisticRegression(max_iter=10000, random_state=42, solver='liblinear').fit(X_train_selected, y_train)
                importances = abs(logistic.coef_[0])
                ranked_features = sorted(zip(selected_features, importances), key=lambda x: x[1], reverse=True)

                # Store selected features and their importance
                all_selected_features.extend([f[0] for f in ranked_features])
                all_importances.extend([f[1] for f in ranked_features])

            # If no features were selected across all folds, return None
            if not all_selected_features:
                return None

            # Aggregate selected features across all folds
            unique_features = list(set(all_selected_features))
            frequency = Counter(all_selected_features)
            avg_importance = {feature: np.mean([imp for feat, imp in zip(all_selected_features, all_importances) if feat == feature])
                              for feature in unique_features}

            # Calculate a composite score for each feature based on frequency and importance
            composite_scores = {feature: 0.5 * (frequency[feature] / loo.get_n_splits(X)) + 0.5 * (avg_importance[feature] / sum(avg_importance.values()))
                                for feature in unique_features}
            sorted_features = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)

            # Evaluate different feature subsets with varying thresholds
            best_overall_performance = 0
            best_overall_setup = {}
            best_percentage = 0
            thresholds = np.linspace(0.05, 0.95, 19)

            for i in range(1, min(len(sorted_features), int(0.8 * len(y))) + 1):
                selected_features = [feature[0] for feature in sorted_features[:i]]
                best_performance_for_features = 0
                best_threshold_for_features = None

                for threshold in thresholds:
                    fold_f1_scores = []

                    # Perform LOOCV prediction with the selected features and model
                    for train_index, test_index in loo.split(X):
                        X_train, X_test = X.iloc[train_index][selected_features], X.iloc[test_index][selected_features]
                        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                        # Use the appropriate model for prediction
                        if ml_model_name == 'XGBoost':
                            model = xgb.XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss', random_state=42)
                        elif ml_model_name == 'RandomForest':
                            model = RandomForestClassifier(n_estimators=300, random_state=42)
                        else:
                            model = ElasticNetCV(cv=5, max_iter=10000, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7)

                        # Fit the model, make predictions and compute F1 score for the current fold
                        model.fit(X_train, y_train)
                        test_prediction = (model.predict(X_test) >= threshold).astype(int) if ml_model_name == 'ElasticNet' else (model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
                        f1_score_current = f1_score(y_test, test_prediction, average='macro')
                        fold_f1_scores.append(f1_score_current)

                    # Calculate the average F1 score for the current threshold
                    f1_score_avg = np.mean(fold_f1_scores)

                    # Record the best performance and threshold for the features
                    if f1_score_avg > best_performance_for_features:
                        best_performance_for_features = f1_score_avg
                        best_threshold_for_features = threshold

                # Update the best overall performance if it improves
                if best_performance_for_features > best_overall_performance:
                    best_overall_performance = best_performance_for_features
                    best_overall_setup = {
                        'features': selected_features,
                        'threshold': best_threshold_for_features,
                        'performance': best_performance_for_features,
                        'feature_selection_method': feature_selector_name,
                        'ml_model': ml_model_name
                    }

            return {
                'feature_selection_method': best_overall_setup['feature_selection_method'],
                'ml_model': best_overall_setup['ml_model'],
                'best_features': best_overall_setup['features'],
                'features_length': len(best_overall_setup['features']),
                'best_threshold': best_overall_setup['threshold'],
                'best_performance': best_overall_setup['performance']
            }

        # Process combinations of feature selection and prediction models
        combinations = [('LASSO', 'ElasticNet'), ('ElasticNet', 'ElasticNet'), ('LASSO', 'XGBoost'), ('ElasticNet', 'XGBoost')]
        results_per_combination = [process_combination(fs, ml) for fs, ml in combinations]
        best_combination = max(results_per_combination, key=lambda x: x['best_performance'] if x is not None else 0)

        return time_point, best_combination

    # Process time points within the specified max_timepoint
    time_points = np.sort(data[data['timepoint_numeric'] < max_timepoint]['timepoint_numeric'].unique())
    results_parallel = Parallel(n_jobs=-1)(delayed(process_time_point)(tp) for tp in time_points)
    results = {tp: result for tp, result in results_parallel if result is not None}

    return results

# Set the max timepoint to consider data before the earliest onset age
max_timepoint = 18

# Evaluate models for all subjects
print("Evaluating Celiac Disease Prediction")
results_ced = evaluate_models(combined_data_filtered, max_timepoint)

# Print results for all subjects
print("Results for Celiac Disease Prediction for All Subjects:")
for time_point, res in results_ced.items():
    if res is not None:
        print(f"Time Point: {time_point}")
        print(f"  Feature Selection Method: {res['feature_selection_method']}")
        print(f"  Machine Learning Model: {res['ml_model']}")
        print(f"  Best F1 Score: {res['best_performance']}")
        print(f"  Best Threshold: {res['best_threshold']}")
        print(f"  Features Used: {res['best_features']}")
        print(f"  Features Length: {res['features_length']}")
        print("-" * 40)

In [ ]:
from sklearn.metrics import f1_score

# Filter data for Celiac disease cases before 18 and 36 months
celiac_data = combined_data_filtered[(combined_data_filtered['Diagnosis'] == 1) & 
                                     (combined_data_filtered['timepoint_numeric'] <= 15)]

# Define early onset vs. late onset based on the thresholds (18 months and 36 months)
celiac_data['Onset_Type'] = np.where(celiac_data['CD_onset'] <= 18, 'Early', 'Late')
celiac_data['Onset_Type'] = np.where(celiac_data['CD_onset'] <= 36, celiac_data['Onset_Type'], 'None')

# Remove rows where Onset_Type is 'None' (for onset after 36 months)
celiac_data = celiac_data[celiac_data['Onset_Type'] != 'None']

# Encode 'Onset_Type' to numerical values: Early = 0, Late = 1
celiac_data['Onset_Type'] = celiac_data['Onset_Type'].map({'Early': 0, 'Late': 1})

# Define selected features for each time point
features_timepoint_12 = [
    'Bacteroides_thetaiotaomicron', 'Bifidobacterium_breve', 'GGB3612_SGB4882', 
    'Bifidobacterium_bifidum', 'Bifidobacterium_adolescentis', 'Bacteroides_caccae', 
    'Eisenbergiella_tayi', 'Phocaeicola_massiliensis', 'Faecalibacterium_prausnitzii', 
    'Ruminococcus_gnavus', 'Veillonella_parvula', 'ARO-PWY: chorismate biosynthesis I', 
    'ARGSYN-PWY: L-arginine biosynthesis I (via L-ornithine)'
]

features_timepoint_15 = [
    'Erysipelatoclostridium_ramosum', 'Bacteroides_fragilis', 'Blautia_producta', 
    'Akkermansia_muciniphila', 'Enterococcus_faecalis', 'Bacteroides_stercoris', 
    'Alistipes_onderdonkii', 'Parabacteroides_distasonis', 'Blautia_wexlerae', 
    'Escherichia_coli', 'Alistipes_finegoldii', 'Bacteroides_thetaiotaomicron', 
    '3-HYDROXYPHENYLACETATE-DEGRADATION-PWY: 4-hydroxyphenylacetate degradation', 
    'ARGININE-SYN4-PWY: L-ornithine biosynthesis II', 'ANAEROFRUCAT-PWY: homolactic fermentation', 
    '1CMET2-PWY: folate transformations III (E. coli)', 'ARGSYN-PWY: L-arginine biosynthesis I (via L-ornithine)', 
    'ARGSYNBSUB-PWY: L-arginine biosynthesis II (acetyl cycle)', 'ARO-PWY: chorismate biosynthesis I', 
    'ARG+POLYAMINE-SYN: superpathway of arginine and polyamine biosynthesis', 'ANAGLYCOLYSIS-PWY: glycolysis III (from glucose)', 
    'AEROBACTINSYN-PWY: aerobactin biosynthesis', 'ARGDEG-PWY: superpathway of L-arginine, putrescine, and 4-aminobutanoate degradation', 
    'ALLANTOINDEG-PWY: superpathway of allantoin degradation in yeast', 'BRANCHED-CHAIN-AA-SYN-PWY: superpathway of branched chain amino acid biosynthesis'
    
]

# Define a function to evaluate combinations of feature selectors and models
def evaluate_models(data):
    results = {}

    # Define a function to process each time point independently
    def process_time_point(time_point):
        print(f"Processing time point: {time_point}")

        # Select features based on the time point
        if time_point == 12:
            selected_features = features_timepoint_12
        elif time_point == 15:
            selected_features = features_timepoint_15
        else:
            print(f"No predefined features for time point {time_point}.")
            return time_point, None

        # Filter data for the current time point and extract selected features
        current_data = data[data['timepoint_numeric'] == time_point]
        X = current_data[selected_features] # Feature matrix
        y = current_data['Onset_Type'] # Labels

        # Ensure there are enough samples to perform LOOCV
        if len(y) < 2:
            print(f"Skipping time point {time_point} due to insufficient samples.")
            return time_point, None

        # Initialize Leave-One-Out cross-validation
        loo = LeaveOneOut()
        best_overall_score = 0
        best_overall_setup = {}

        # Define a function to process each feature selector and model combination
        def process_combination(feature_selector_name, ml_model_name):
            all_selected_features = []
            all_importances = []

            # Loop through the training/test splits generated by LOOCV
            for train_index, test_index in loo.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # Perform feature selection based on the specified selector
                if feature_selector_name == 'LASSO':
                    feature_selector = LassoCV(cv=5, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30)).fit(X_train, y_train)
                else:
                    feature_selector = ElasticNetCV(cv=5, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7).fit(X_train, y_train)

                # Select features with non-zero coefficients
                selected_features = X_train.columns[feature_selector.coef_ != 0]
                selected_features = selected_features[:min(len(selected_features), int(0.8 * len(y_train)))]

                # If no features are selected, skip this iteration
                if len(selected_features) == 0:
                    continue

                # Select the relevant features from the training set
                X_train_selected = X_train[selected_features]

                # Perform logistic regression for ranking the selected features based on importance
                logistic = LogisticRegression(max_iter=10000, random_state=42, solver='liblinear').fit(X_train_selected, y_train)
                importances = abs(logistic.coef_[0])
                ranked_features = sorted(zip(selected_features, importances), key=lambda x: x[1], reverse=True)

                # Store selected features and their importance
                all_selected_features.extend([f[0] for f in ranked_features])
                all_importances.extend([f[1] for f in ranked_features])

            # If no features were selected across all folds, return None
            if not all_selected_features:
                return None

            # Aggregate selected features across all folds
            unique_features = list(set(all_selected_features))
            frequency = Counter(all_selected_features)
            avg_importance = {feature: np.mean([imp for feat, imp in zip(all_selected_features, all_importances) if feat == feature])
                              for feature in unique_features}

            # Calculate a composite score for each feature based on frequency and importance
            composite_scores = {feature: 0.5 * (frequency[feature] / loo.get_n_splits(X)) + 0.5 * (avg_importance[feature] / sum(avg_importance.values()))
                                for feature in unique_features}
            sorted_features = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)

            # Evaluate different feature subsets with varying thresholds
            best_overall_performance = 0
            best_overall_setup = {}
            best_percentage = 0
            thresholds = np.linspace(0.05, 0.95, 19)

            for i in range(1, min(len(sorted_features), int(0.8 * len(y))) + 1):
                selected_features = [feature[0] for feature in sorted_features[:i]]
                best_performance_for_features = 0
                best_threshold_for_features = None

                for threshold in thresholds:
                    fold_f1_scores = []

                    # Perform LOOCV prediction with the selected features and model
                    for train_index, test_index in loo.split(X):
                        X_train, X_test = X.iloc[train_index][selected_features], X.iloc[test_index][selected_features]
                        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                        # Use the appropriate model for prediction
                        if ml_model_name == 'XGBoost':
                            model = xgb.XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss', random_state=42)
                        elif ml_model_name == 'RandomForest':
                            model = RandomForestClassifier(n_estimators=300, random_state=42)
                        else:
                            model = ElasticNetCV(cv=5, max_iter=10000, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7)

                        # Fit the model, make predictions and compute F1 score for the current fold
                        model.fit(X_train, y_train)
                        test_prediction = (model.predict(X_test) >= threshold).astype(int) if ml_model_name == 'ElasticNet' else (model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
                        f1_score_current = f1_score(y_test, test_prediction, average='macro')
                        fold_f1_scores.append(f1_score_current)

                    # Calculate the average F1 score for the current threshold
                    f1_score_avg = np.mean(fold_f1_scores)

                    # Record the best performance and threshold for the features
                    if f1_score_avg > best_performance_for_features:
                        best_performance_for_features = f1_score_avg
                        best_threshold_for_features = threshold

                # Update the best overall performance if it improves
                if best_performance_for_features > best_overall_performance:
                    best_overall_performance = best_performance_for_features
                    best_overall_setup = {
                        'features': selected_features,
                        'threshold': best_threshold_for_features,
                        'performance': best_performance_for_features,
                        'feature_selection_method': feature_selector_name,
                        'ml_model': ml_model_name
                    }

            return {
                'feature_selection_method': best_overall_setup['feature_selection_method'],
                'ml_model': best_overall_setup['ml_model'],
                'best_features': best_overall_setup['features'],
                'features_length': len(best_overall_setup['features']),
                'best_threshold': best_overall_setup['threshold'],
                'best_performance': best_overall_setup['performance']
            }

        # Process combinations of feature selection and prediction models
        combinations = [('LASSO', 'ElasticNet'), ('ElasticNet', 'ElasticNet'), ('LASSO', 'XGBoost'), ('ElasticNet', 'XGBoost')]
        results_per_combination = [process_combination(fs, ml) for fs, ml in combinations]
        best_combination = max(results_per_combination, key=lambda x: x['best_performance'] if x is not None else 0)

        return time_point, best_combination

    # Process time points within the specified range (before 15 months)
    time_points = np.sort(data['timepoint_numeric'].unique())
    results_parallel = Parallel(n_jobs=-1)(delayed(process_time_point)(tp) for tp in time_points)
    results = {tp: result for tp, result in results_parallel if result is not None}

    return results

# Evaluate the model
results_early_late = evaluate_models(celiac_data)

# Print results
print("Results for Early Onset vs. Late Onset Prediction:")
for time_point, res in results_early_late.items():
    if res is not None:
        print(f"Time Point: {time_point}")
        print(f"  Feature Selection Method: {res['feature_selection_method']}")
        print(f"  Machine Learning Model: {res['ml_model']}")
        print(f"  Best F1 Score: {res['best_performance']}")
        print(f"  Best Threshold: {res['best_threshold']}")
        print(f"  Features Used: {res['best_features']}")
        print(f"  Features Length: {res['features_length']}")
        print("-" * 40)

In [ ]:
from sklearn.metrics import f1_score

# Filter data to include only relative time points from -18 to onset
filtered_data = combined_data_filtered[(combined_data_filtered['Relative_timepoint'] >= -18) & (combined_data_filtered['Relative_timepoint'] <= 0)]

# Define selected features for each time point
features_timepoint_18 = [
    'Candidatus_Cibionibacter_quicibialis', 'Anaerobutyricum_soehngenii', 'Clostridium_SGB6173', 'Bacteroides_ovatus', 
    'Bifidobacterium_adolescentis', 'Dysosmobacter_welbionis', 'Bacteroides_fragilis', 'Bifidobacterium_catenulatum', 
    'Bifidobacterium_pseudocatenulatum', 'Blautia_wexlerae', 'Clostridiales_bacterium_KLE1615', 'Bacteroides_thetaiotaomicron', 
    'Coprococcus_comes', 'Eisenbergiella_tayi', 'Bacteroides_stercoris', 'Lachnospiraceae_bacterium', 'Escherichia_coli', 
    'Roseburia_faecis', 'Mediterraneibacter_faecis', 'Phascolarctobacterium_faecium', 'Flavonifractor_plautii', 
    'Roseburia_intestinalis', 'Anaerobutyricum_hallii', 'Bacteroides_cellulosilyticus', 'Roseburia_sp_AF02_12', 
    'Clostridium_sp_AM33_3', 'Lachnospira_eligens', 'Clostridium_SGB6179', 'Ruminococcus_torques', 'Ruminococcus_callidus', 
    'Streptococcus_thermophilus', 'FUCCAT-PWY: fucose degradation', 'P125-PWY: superpathway of (R,R)-butanediol biosynthesis', 
    'FUC-RHAMCAT-PWY: superpathway of fucose and rhamnose degradation', 'POLYAMSYN-PWY: superpathway of polyamine biosynthesis I', 
    'ARG+POLYAMINE-SYN: superpathway of arginine and polyamine biosynthesis', 'KETOGLUCONMET-PWY: ketogluconate metabolism', 
    'METH-ACETATE-PWY: methanogenesis from acetate', 'GALACTARDEG-PWY: D-galactarate degradation I', 
    'POLYISOPRENSYN-PWY: polyisoprenoid biosynthesis (E. coli)', 'PWY-5367: petroselinate biosynthesis', 
    'PWY-5022: 4-aminobutanoate degradation V', 'GLUCUROCAT-PWY: superpathway of &beta;-D-glucuronosides degradation', 
    'BIOTIN-BIOSYNTHESIS-PWY: biotin biosynthesis I'
]

features_timepoint_15 = [
    'Bacteroides_thetaiotaomicron', 'Bifidobacterium_bifidum', 'Anaerobutyricum_hallii', 'Bifidobacterium_adolescentis', 
    'Bifidobacterium_pseudocatenulatum', 'Alistipes_onderdonkii', 'Alistipes_putredinis', 'Akkermansia_muciniphila', 
    'Clostridiaceae_bacterium_Marseille_Q3526', 'KETOGLUCONMET-PWY: ketogluconate metabolism', 'PPGPPMET-PWY: ppGpp metabolism', 
    'P105-PWY: TCA cycle IV (2-oxoglutarate decarboxylase)', 'GALACTITOLCAT-PWY: galactitol degradation'
]
features_timepoint_12 = [
    'Anaerobutyricum_hallii', 'GGB9469_SGB14862', 'Dialister_invisus', 'Alistipes_finegoldii', 'Alistipes_onderdonkii', 
    'Enterocloster_bolteae', 'GGB3740_SGB5076', 'Enterococcus_faecium', 'Eggerthella_lenta', 'Alistipes_putredinis', 
    'Bacteroides_stercoris', 'Blautia_wexlerae', 'Bacteroides_fragilis', 'Enterocloster_clostridioformis', 
    'Intestinibacter_bartlettii', 'Anaerostipes_hadrus', 'PWY-5130: 2-oxobutanoate degradation I', 
    'PWY-5690: TCA cycle II (plants and fungi)', 'ALLANTOINDEG-PWY: superpathway of allantoin degradation in yeast', 
    'PWY-6163: chorismate biosynthesis from 3-dehydroquinate', 'P108-PWY: pyruvate fermentation to propanoate I', 
    'P4-PWY: superpathway of L-lysine, L-threonine and L-methionine biosynthesis I', 
    'COA-PWY-1: superpathway of coenzyme A biosynthesis III (mammals)', 
    'GOLPDLCAT-PWY: superpathway of glycerol degradation to 1,3-propanediol', 'PPGPPMET-PWY: ppGpp metabolism', 
    'FASYN-INITIAL-PWY: superpathway of fatty acid biosynthesis initiation (E. coli)', 
    'HEXITOLDEGSUPER-PWY: superpathway of hexitol degradation (bacteria)', 
    'PWY-6121: 5-aminoimidazole ribonucleotide biosynthesis I', 
    'PEPTIDOGLYCANSYN-PWY: peptidoglycan biosynthesis I (meso-diaminopimelate containing)', 
    'COMPLETE-ARO-PWY: superpathway of aromatic amino acid biosynthesis', 
    'METHGLYUT-PWY: superpathway of methylglyoxal degradation', 'PWY-3841: folate transformations II (plants)', 
    'P23-PWY: reductive TCA cycle I', 'PWY-241: C4 photosynthetic carbon assimilation cycle, NADP-ME type', 
    'HISTSYN-PWY: L-histidine biosynthesis', 
    'GLYCOLYSIS-TCA-GLYOX-BYPASS: superpathway of glycolysis, pyruvate dehydrogenase, TCA, and glyoxylate bypass', 
    'PWY-1861: formaldehyde assimilation II (assimilatory RuMP Cycle)'
]

features_timepoint_9 = [
    'Bacteroides_ovatus', 'Bifidobacterium_catenulatum', 'Clostridiales_bacterium', 'Bifidobacterium_adolescentis', 
    'Blautia_wexlerae', 'Candidatus_Cibionibacter_quicibialis', 'Clostridium_SGB6173', 'Bacteroides_thetaiotaomicron', 
    'Bacteroides_xylanisolvens', 'Dialister_invisus', 'Bacteroides_uniformis', 'HSERMETANA-PWY: L-methionine biosynthesis III', 
    'COMPLETE-ARO-PWY: superpathway of aromatic amino acid biosynthesis', 
    'HCAMHPDEG-PWY: 3-phenylpropanoate and 3-(3-hydroxyphenyl)propanoate degradation to 2-hydroxypentadienoate', 
    'POLYISOPRENSYN-PWY: polyisoprenoid biosynthesis (E. coli)', 'DAPLYSINESYN-PWY: L-lysine biosynthesis I', 
    'GALACT-GLUCUROCAT-PWY: superpathway of hexuronide and hexuronate degradation', 
    'NAGLIPASYN-PWY: lipid IVA biosynthesis (E. coli)', 'FOLSYN-PWY: superpathway of tetrahydrofolate biosynthesis and salvage', 
    'GOLPDLCAT-PWY: superpathway of glycerol degradation to 1,3-propanediol', 
    'ARGSYNBSUB-PWY: L-arginine biosynthesis II (acetyl cycle)', 
    'PRPP-PWY: superpathway of histidine, purine, and pyrimidine biosynthesis'
]
features_timepoint_6 = [
    'GGB9480_SGB14874', 'Faecalibacillus_intestinalis', 'Alistipes_putredinis', 'Brotolimicola_acetigignens', 
    'Dorea_formicigenerans', 'Anaerostipes_hadrus', 'Eisenbergiella_massiliensis', 'Phascolarctobacterium_faecium', 
    'Bacteroides_thetaiotaomicron', 'Clostridiales_bacterium_KLE1615', 'Blautia_hansenii', 'Roseburia_hominis', 
    'Enterococcus_faecalis', 'Clostridium_saudiense', 'Bifidobacterium_longum', 'Bifidobacterium_dentium', 
    'Streptococcus_thermophilus', 'PWY-6606: guanosine nucleotides degradation II', 
    'PWY-6353: purine nucleotides degradation II (aerobic)'
]

features_timepoint_3 = [
    'Gemmiger_formicilis', 'Lachnospira_pectinoschiza', 'Bacteroides_caccae', 'GGB3256_SGB4303', 'Romboutsia_timonensis', 
    'Parabacteroides_merdae', 'Enterococcus_faecium', 'Enterococcus_faecalis', 'Clostridium_sp_C5_48', 'Phocaeicola_vulgatus', 
    'Clostridium_SGB6173', 'Escherichia_coli', 'Roseburia_faecis', 'Dorea_longicatena', 'Bifidobacterium_bifidum', 
    'Bifidobacterium_catenulatum', 'Bifidobacterium_animalis', 'Roseburia_intestinalis', 'Tyzzerella_nexilis', 
    'Veillonella_ratti', 'P185-PWY: formaldehyde assimilation III (dihydroxyacetone cycle)', 
    'PWY-5180: toluene degradation I (aerobic) (via o-cresol)', 
    'PROPFERM-PWY: superpathway of L-alanine fermentation (Stickland reaction)', 
    'PWY-5138: fatty acid &beta;-oxidation IV (unsaturated, even number)', 
    'GLYCOLYSIS-TCA-GLYOX-BYPASS: superpathway of glycolysis, pyruvate dehydrogenase, TCA, and glyoxylate bypass', 
    "PWY-6123: inosine-5'-phosphate biosynthesis I", 'CENTFERM-PWY: pyruvate fermentation to butanoate', 
    'PWY-6285: superpathway of fatty acids biosynthesis (E. coli)', "PWY-6124: inosine-5'-phosphate biosynthesis II", 
    'PWY-1861: formaldehyde assimilation II (assimilatory RuMP Cycle)', 
    'ASPASN-PWY: superpathway of L-aspartate and L-asparagine biosynthesis', 
    'PWY-6470: peptidoglycan biosynthesis V (&beta;-lactam resistance)', 
    'COLANSYN-PWY: colanic acid building blocks biosynthesis', 'DTDPRHAMSYN-PWY: dTDP-&beta;-L-rhamnose biosynthesis'
]
features_timepoint_0 = [
    'Eubacterium_siraeum', 'Eisenbergiella_massiliensis', 'Alistipes_putredinis', 'Clostridium_SGB6179', 'Enterocloster_bolteae', 
    'GGB9469_SGB14862', 'GGB51647_SGB4348', 'GGB9480_SGB14874', 'Clostridiales_bacterium_KLE1615', 
    'Erysipelatoclostridium_ramosum', 'Alistipes_onderdonkii', 'Parasutterella_excrementihominis', 
    'Fusicatenibacter_saccharivorans', 'Blautia_wexlerae', 'Bifidobacterium_pseudocatenulatum', 'Parabacteroides_distasonis', 
    'Bifidobacterium_adolescentis', 'Bifidobacterium_longum', 'Phocaeicola_vulgatus', 'GGB3256_SGB4303', 
    'Clostridiaceae_bacterium', 'Roseburia_faecis', 'Lacrimispora_amygdalina', 'GGB4456_SGB6141', 'Blautia_obeum', 
    'Anaerostipes_hadrus', 'Bifidobacterium_breve', 'Candidatus_Cibionibacter_quicibialis', 'Brotolimicola_acetigignens', 
    'Roseburia_intestinalis', 'Enterococcus_faecalis', 'Blautia_faecis', 'Clostridium_leptum', 'Clostridium_sp_AT4', 
    'Clostridium_SGB4750', 'Coprococcus_eutactus', 'GGB9534_SGB14937', 'Intestinimonas_butyriciproducens', 
    'Lachnospira_sp_NSJ_43', 'Clostridium_sp_C5_48', 'Fusicatenibacter_sp_CLA_AA_H277', 'Clostridium_sp_AM22_11AC', 
    'Anaerostipes_caccae', 'Pseudoflavonifractor_capillosus', 'Roseburia_hominis', 'Ruminococcus_bromii', 
    'Oscillibacter_valericigenes', 'Flavonifractor_plautii', 'Ruminococcus_gnavus', 'PWY-4041: &gamma;-glutamyl cycle', 
    'PWY-6549: L-glutamine biosynthesis III', 'FUC-RHAMCAT-PWY: superpathway of fucose and rhamnose degradation', 
    'PWY-5677: succinate fermentation to butanoate', 'P163-PWY: L-lysine fermentation to acetate and butanoate'
]

# Define a function to evaluate combinations of feature selectors and models
def evaluate_models(data):
    results = {}

    # Define a function to process each time point independently
    def process_time_point(time_point):
        print(f"Processing relative time point: {time_point}")

        # Select features based on the time point
        if time_point == -18:
            selected_features = features_timepoint_18
        elif time_point == -15:
            selected_features = features_timepoint_15
        elif time_point == -12:
            selected_features = features_timepoint_12
        elif time_point == -9:
            selected_features = features_timepoint_9
        elif time_point == -6:
            selected_features = features_timepoint_6
        elif time_point == -3:
            selected_features = features_timepoint_3
        elif time_point == 0:
            selected_features = features_timepoint_0
        else:
            print(f"No predefined features for time point {time_point}.")
            return time_point, None

        # Filter data for the current relative time point and extract selected features
        current_data = data[data['Relative_timepoint'] == time_point]
        X = current_data[selected_features] # Feature matrix
        y = current_data['Diagnosis'] # Labels

        # Ensure there are enough samples to perform LOOCV
        if len(y) < 2:
            print(f"Skipping relative time point {time_point} due to insufficient samples.")
            return time_point, None

        # Initialize Leave-One-Out cross-validation
        loo = LeaveOneOut()
        best_overall_score = 0
        best_overall_setup = {}

        # Define a function to process each feature selector and model combination
        def process_combination(feature_selector_name, ml_model_name):
            all_selected_features = []
            all_importances = []

            # Loop through the training/test splits generated by LOOCV
            for train_index, test_index in loo.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # Perform feature selection based on the specified selector
                if feature_selector_name == 'LASSO':
                    feature_selector = LassoCV(cv=loo, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30)).fit(X_train, y_train)
                else:
                    feature_selector = ElasticNetCV(cv=loo, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7).fit(X_train, y_train)

                # Select features with non-zero coefficients
                selected_features = X_train.columns[feature_selector.coef_ != 0]
                selected_features = selected_features[:min(len(selected_features), int(0.8 * len(y_train)))]

                # If no features are selected, skip this iteration
                if len(selected_features) == 0:
                    continue

                # Select the relevant features from the training set
                X_train_selected = X_train[selected_features]

                # Perform logistic regression for ranking the selected features based on importance
                logistic = LogisticRegression(max_iter=10000, random_state=42, solver='liblinear').fit(X_train_selected, y_train)
                importances = abs(logistic.coef_[0])
                ranked_features = sorted(zip(selected_features, importances), key=lambda x: x[1], reverse=True)

                # Store selected features and their importance
                all_selected_features.extend([f[0] for f in ranked_features])
                all_importances.extend([f[1] for f in ranked_features])

            # If no features were selected across all folds, return None
            if not all_selected_features:
                return None

            # Aggregate selected features across all folds
            unique_features = list(set(all_selected_features))
            frequency = Counter(all_selected_features)
            avg_importance = {feature: np.mean([imp for feat, imp in zip(all_selected_features, all_importances) if feat == feature])
                              for feature in unique_features}

            # Calculate a composite score for each feature based on frequency and importance
            composite_scores = {feature: 0.5 * (frequency[feature] / loo.get_n_splits(X)) + 0.5 * (avg_importance[feature] / sum(avg_importance.values()))
                                for feature in unique_features}
            sorted_features = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)

            # Evaluate different feature subsets with varying thresholds
            best_overall_performance = 0
            best_overall_setup = {}
            best_percentage = 0
            thresholds = np.linspace(0.05, 0.95, 19)

            for i in range(1, min(len(sorted_features), int(0.8 * len(y))) + 1):
                selected_features = [feature[0] for feature in sorted_features[:i]]
                best_performance_for_features = 0
                best_threshold_for_features = None

                for threshold in thresholds:
                    fold_f1_scores = []

                    # Perform LOOCV prediction with the selected features and model
                    for train_index, test_index in loo.split(X):
                        X_train, X_test = X.iloc[train_index][selected_features], X.iloc[test_index][selected_features]
                        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                        # Use the appropriate model for prediction
                        if ml_model_name == 'XGBoost':
                            model = xgb.XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss', random_state=42)
                        elif ml_model_name == 'RandomForest':
                            model = RandomForestClassifier(n_estimators=300, random_state=42)
                        else:
                            model = ElasticNetCV(cv=5, max_iter=10000, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7)

                        # Fit the model, make predictions and compute F1 score for the current fold
                        model.fit(X_train, y_train)
                        test_prediction = (model.predict(X_test) >= threshold).astype(int) if ml_model_name == 'ElasticNet' else (model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
                        f1_score_current = f1_score(y_test, test_prediction, average='macro')
                        fold_f1_scores.append(f1_score_current)

                    # Calculate the average F1 score for the current threshold
                    f1_score_avg = np.mean(fold_f1_scores)

                    # Record the best performance and threshold for the features
                    if f1_score_avg > best_performance_for_features:
                        best_performance_for_features = f1_score_avg
                        best_threshold_for_features = threshold

                # Update the best overall performance if it improves
                if best_performance_for_features > best_overall_performance:
                    best_overall_performance = best_performance_for_features
                    best_overall_setup = {
                        'features': selected_features,
                        'threshold': best_threshold_for_features,
                        'performance': best_performance_for_features,
                        'feature_selection_method': feature_selector_name,
                        'ml_model': ml_model_name
                    }

            return {
                'feature_selection_method': best_overall_setup['feature_selection_method'],
                'ml_model': best_overall_setup['ml_model'],
                'best_features': best_overall_setup['features'],
                'features_length': len(best_overall_setup['features']),
                'best_threshold': best_overall_setup['threshold'],
                'best_performance': best_overall_setup['performance']
            }

        # Process combinations of feature selection and prediction models
        combinations = [('LASSO', 'ElasticNet'), ('ElasticNet', 'ElasticNet')]
        results_per_combination = [process_combination(fs, ml) for fs, ml in combinations]
        best_combination = max(results_per_combination, key=lambda x: x['best_performance'] if x is not None else 0)

        return time_point, best_combination

    # Process relative time points within the specified range
    time_points = np.sort(data['Relative_timepoint'].unique())
    results_parallel = Parallel(n_jobs=-1)(delayed(process_time_point)(tp) for tp in time_points)
    results = {tp: result for tp, result in results_parallel if result is not None}

    return results

# Evaluate the model
results_relative = evaluate_models(filtered_data)

# Print results
print("Results for Celiac Disease Prediction by Relative Time Point:")
for time_point, res in results_relative.items():
    if res is not None:
        print(f"Relative Time Point: {time_point}")
        print(f"  Feature Selection Method: {res['feature_selection_method']}")
        print(f"  Machine Learning Model: {res['ml_model']}")
        print(f"  Best F1 Score: {res['best_performance']}")
        print(f"  Best Threshold: {res['best_threshold']}")
        print(f"  Features Used: {res['best_features']}")
        print(f"  Features Length: {res['features_length']}")
        print("-" * 40)

In [ ]:
from sklearn.metrics import f1_score

# Filter data to include only relative time points from -18 to onset
filtered_data = combined_data_filtered[(combined_data_filtered['Relative_timepoint'] >= -18) & (combined_data_filtered['Relative_timepoint'] <= 0)]

# Define selected features for each time point
features_timepoint_18 = [
    'Candidatus_Cibionibacter_quicibialis', 'Anaerobutyricum_soehngenii', 'Clostridium_SGB6173', 'Bacteroides_ovatus', 
    'Bifidobacterium_adolescentis', 'Dysosmobacter_welbionis', 'Bacteroides_fragilis', 'Bifidobacterium_catenulatum', 
    'Bifidobacterium_pseudocatenulatum', 'Blautia_wexlerae', 'Clostridiales_bacterium_KLE1615', 'Bacteroides_thetaiotaomicron', 
    'Coprococcus_comes', 'Eisenbergiella_tayi', 'Bacteroides_stercoris', 'Lachnospiraceae_bacterium', 'Escherichia_coli', 
    'Roseburia_faecis', 'Mediterraneibacter_faecis', 'Phascolarctobacterium_faecium', 'Flavonifractor_plautii', 
    'Roseburia_intestinalis', 'Anaerobutyricum_hallii', 'Bacteroides_cellulosilyticus', 'Roseburia_sp_AF02_12', 
    'Clostridium_sp_AM33_3', 'Lachnospira_eligens', 'Clostridium_SGB6179', 'Ruminococcus_torques', 'Ruminococcus_callidus', 
    'Streptococcus_thermophilus', 'FUCCAT-PWY: fucose degradation', 'P125-PWY: superpathway of (R,R)-butanediol biosynthesis', 
    'FUC-RHAMCAT-PWY: superpathway of fucose and rhamnose degradation', 'POLYAMSYN-PWY: superpathway of polyamine biosynthesis I', 
    'ARG+POLYAMINE-SYN: superpathway of arginine and polyamine biosynthesis', 'KETOGLUCONMET-PWY: ketogluconate metabolism', 
    'METH-ACETATE-PWY: methanogenesis from acetate', 'GALACTARDEG-PWY: D-galactarate degradation I', 
    'POLYISOPRENSYN-PWY: polyisoprenoid biosynthesis (E. coli)', 'PWY-5367: petroselinate biosynthesis', 
    'PWY-5022: 4-aminobutanoate degradation V', 'GLUCUROCAT-PWY: superpathway of &beta;-D-glucuronosides degradation', 
    'BIOTIN-BIOSYNTHESIS-PWY: biotin biosynthesis I'
]

features_timepoint_15 = [
    'Bacteroides_thetaiotaomicron', 'Bifidobacterium_bifidum', 'Anaerobutyricum_hallii', 'Bifidobacterium_adolescentis', 
    'Bifidobacterium_pseudocatenulatum', 'Alistipes_onderdonkii', 'Alistipes_putredinis', 'Akkermansia_muciniphila', 
    'Clostridiaceae_bacterium_Marseille_Q3526', 'KETOGLUCONMET-PWY: ketogluconate metabolism', 'PPGPPMET-PWY: ppGpp metabolism', 
    'P105-PWY: TCA cycle IV (2-oxoglutarate decarboxylase)', 'GALACTITOLCAT-PWY: galactitol degradation'
]
features_timepoint_12 = [
    'Anaerobutyricum_hallii', 'GGB9469_SGB14862', 'Dialister_invisus', 'Alistipes_finegoldii', 'Alistipes_onderdonkii', 
    'Enterocloster_bolteae', 'GGB3740_SGB5076', 'Enterococcus_faecium', 'Eggerthella_lenta', 'Alistipes_putredinis', 
    'Bacteroides_stercoris', 'Blautia_wexlerae', 'Bacteroides_fragilis', 'Enterocloster_clostridioformis', 
    'Intestinibacter_bartlettii', 'Anaerostipes_hadrus', 'PWY-5130: 2-oxobutanoate degradation I', 
    'PWY-5690: TCA cycle II (plants and fungi)', 'ALLANTOINDEG-PWY: superpathway of allantoin degradation in yeast', 
    'PWY-6163: chorismate biosynthesis from 3-dehydroquinate', 'P108-PWY: pyruvate fermentation to propanoate I', 
    'P4-PWY: superpathway of L-lysine, L-threonine and L-methionine biosynthesis I', 
    'COA-PWY-1: superpathway of coenzyme A biosynthesis III (mammals)', 
    'GOLPDLCAT-PWY: superpathway of glycerol degradation to 1,3-propanediol', 'PPGPPMET-PWY: ppGpp metabolism', 
    'FASYN-INITIAL-PWY: superpathway of fatty acid biosynthesis initiation (E. coli)', 
    'HEXITOLDEGSUPER-PWY: superpathway of hexitol degradation (bacteria)', 
    'PWY-6121: 5-aminoimidazole ribonucleotide biosynthesis I', 
    'PEPTIDOGLYCANSYN-PWY: peptidoglycan biosynthesis I (meso-diaminopimelate containing)', 
    'COMPLETE-ARO-PWY: superpathway of aromatic amino acid biosynthesis', 
    'METHGLYUT-PWY: superpathway of methylglyoxal degradation', 'PWY-3841: folate transformations II (plants)', 
    'P23-PWY: reductive TCA cycle I', 'PWY-241: C4 photosynthetic carbon assimilation cycle, NADP-ME type', 
    'HISTSYN-PWY: L-histidine biosynthesis', 
    'GLYCOLYSIS-TCA-GLYOX-BYPASS: superpathway of glycolysis, pyruvate dehydrogenase, TCA, and glyoxylate bypass', 
    'PWY-1861: formaldehyde assimilation II (assimilatory RuMP Cycle)'
]

features_timepoint_9 = [
    'Bacteroides_ovatus', 'Bifidobacterium_catenulatum', 'Clostridiales_bacterium', 'Bifidobacterium_adolescentis', 
    'Blautia_wexlerae', 'Candidatus_Cibionibacter_quicibialis', 'Clostridium_SGB6173', 'Bacteroides_thetaiotaomicron', 
    'Bacteroides_xylanisolvens', 'Dialister_invisus', 'Bacteroides_uniformis', 'HSERMETANA-PWY: L-methionine biosynthesis III', 
    'COMPLETE-ARO-PWY: superpathway of aromatic amino acid biosynthesis', 
    'HCAMHPDEG-PWY: 3-phenylpropanoate and 3-(3-hydroxyphenyl)propanoate degradation to 2-hydroxypentadienoate', 
    'POLYISOPRENSYN-PWY: polyisoprenoid biosynthesis (E. coli)', 'DAPLYSINESYN-PWY: L-lysine biosynthesis I', 
    'GALACT-GLUCUROCAT-PWY: superpathway of hexuronide and hexuronate degradation', 
    'NAGLIPASYN-PWY: lipid IVA biosynthesis (E. coli)', 'FOLSYN-PWY: superpathway of tetrahydrofolate biosynthesis and salvage', 
    'GOLPDLCAT-PWY: superpathway of glycerol degradation to 1,3-propanediol', 
    'ARGSYNBSUB-PWY: L-arginine biosynthesis II (acetyl cycle)', 
    'PRPP-PWY: superpathway of histidine, purine, and pyrimidine biosynthesis'
]
features_timepoint_6 = [
    'GGB9480_SGB14874', 'Faecalibacillus_intestinalis', 'Alistipes_putredinis', 'Brotolimicola_acetigignens', 
    'Dorea_formicigenerans', 'Anaerostipes_hadrus', 'Eisenbergiella_massiliensis', 'Phascolarctobacterium_faecium', 
    'Bacteroides_thetaiotaomicron', 'Clostridiales_bacterium_KLE1615', 'Blautia_hansenii', 'Roseburia_hominis', 
    'Enterococcus_faecalis', 'Clostridium_saudiense', 'Bifidobacterium_longum', 'Bifidobacterium_dentium', 
    'Streptococcus_thermophilus', 'PWY-6606: guanosine nucleotides degradation II', 
    'PWY-6353: purine nucleotides degradation II (aerobic)'
]

features_timepoint_3 = [
    'Gemmiger_formicilis', 'Lachnospira_pectinoschiza', 'Bacteroides_caccae', 'GGB3256_SGB4303', 'Romboutsia_timonensis', 
    'Parabacteroides_merdae', 'Enterococcus_faecium', 'Enterococcus_faecalis', 'Clostridium_sp_C5_48', 'Phocaeicola_vulgatus', 
    'Clostridium_SGB6173', 'Escherichia_coli', 'Roseburia_faecis', 'Dorea_longicatena', 'Bifidobacterium_bifidum', 
    'Bifidobacterium_catenulatum', 'Bifidobacterium_animalis', 'Roseburia_intestinalis', 'Tyzzerella_nexilis', 
    'Veillonella_ratti', 'P185-PWY: formaldehyde assimilation III (dihydroxyacetone cycle)', 
    'PWY-5180: toluene degradation I (aerobic) (via o-cresol)', 
    'PROPFERM-PWY: superpathway of L-alanine fermentation (Stickland reaction)', 
    'PWY-5138: fatty acid &beta;-oxidation IV (unsaturated, even number)', 
    'GLYCOLYSIS-TCA-GLYOX-BYPASS: superpathway of glycolysis, pyruvate dehydrogenase, TCA, and glyoxylate bypass', 
    "PWY-6123: inosine-5'-phosphate biosynthesis I", 'CENTFERM-PWY: pyruvate fermentation to butanoate', 
    'PWY-6285: superpathway of fatty acids biosynthesis (E. coli)', "PWY-6124: inosine-5'-phosphate biosynthesis II", 
    'PWY-1861: formaldehyde assimilation II (assimilatory RuMP Cycle)', 
    'ASPASN-PWY: superpathway of L-aspartate and L-asparagine biosynthesis', 
    'PWY-6470: peptidoglycan biosynthesis V (&beta;-lactam resistance)', 
    'COLANSYN-PWY: colanic acid building blocks biosynthesis', 'DTDPRHAMSYN-PWY: dTDP-&beta;-L-rhamnose biosynthesis'
]
features_timepoint_0 = [
    'Eubacterium_siraeum', 'Eisenbergiella_massiliensis', 'Alistipes_putredinis', 'Clostridium_SGB6179', 'Enterocloster_bolteae', 
    'GGB9469_SGB14862', 'GGB51647_SGB4348', 'GGB9480_SGB14874', 'Clostridiales_bacterium_KLE1615', 
    'Erysipelatoclostridium_ramosum', 'Alistipes_onderdonkii', 'Parasutterella_excrementihominis', 
    'Fusicatenibacter_saccharivorans', 'Blautia_wexlerae', 'Bifidobacterium_pseudocatenulatum', 'Parabacteroides_distasonis', 
    'Bifidobacterium_adolescentis', 'Bifidobacterium_longum', 'Phocaeicola_vulgatus', 'GGB3256_SGB4303', 
    'Clostridiaceae_bacterium', 'Roseburia_faecis', 'Lacrimispora_amygdalina', 'GGB4456_SGB6141', 'Blautia_obeum', 
    'Anaerostipes_hadrus', 'Bifidobacterium_breve', 'Candidatus_Cibionibacter_quicibialis', 'Brotolimicola_acetigignens', 
    'Roseburia_intestinalis', 'Enterococcus_faecalis', 'Blautia_faecis', 'Clostridium_leptum', 'Clostridium_sp_AT4', 
    'Clostridium_SGB4750', 'Coprococcus_eutactus', 'GGB9534_SGB14937', 'Intestinimonas_butyriciproducens', 
    'Lachnospira_sp_NSJ_43', 'Clostridium_sp_C5_48', 'Fusicatenibacter_sp_CLA_AA_H277', 'Clostridium_sp_AM22_11AC', 
    'Anaerostipes_caccae', 'Pseudoflavonifractor_capillosus', 'Roseburia_hominis', 'Ruminococcus_bromii', 
    'Oscillibacter_valericigenes', 'Flavonifractor_plautii', 'Ruminococcus_gnavus', 'PWY-4041: &gamma;-glutamyl cycle', 
    'PWY-6549: L-glutamine biosynthesis III', 'FUC-RHAMCAT-PWY: superpathway of fucose and rhamnose degradation', 
    'PWY-5677: succinate fermentation to butanoate', 'P163-PWY: L-lysine fermentation to acetate and butanoate'
]

# Define a function to evaluate combinations of feature selectors and models
def evaluate_models(data):
    results = {}

    # Define a function to process each time point independently
    def process_time_point(time_point):
        print(f"Processing relative time point: {time_point}")

        # Select features based on the time point
        if time_point == -18:
            selected_features = features_timepoint_18
        elif time_point == -15:
            selected_features = features_timepoint_15
        elif time_point == -12:
            selected_features = features_timepoint_12
        elif time_point == -9:
            selected_features = features_timepoint_9
        elif time_point == -6:
            selected_features = features_timepoint_6
        elif time_point == -3:
            selected_features = features_timepoint_3
        elif time_point == 0:
            selected_features = features_timepoint_0
        else:
            print(f"No predefined features for time point {time_point}.")
            return time_point, None

        # Filter data for the current relative time point and extract selected features
        current_data = data[data['Relative_timepoint'] == time_point]
        X = current_data[selected_features] # Feature matrix
        y = current_data['Diagnosis'] # Labels

        # Ensure there are enough samples to perform LOOCV
        if len(y) < 2:
            print(f"Skipping relative time point {time_point} due to insufficient samples.")
            return time_point, None

        # Initialize Leave-One-Out cross-validation
        loo = LeaveOneOut()
        best_overall_score = 0
        best_overall_setup = {}

        # Define a function to process each feature selector and model combination
        def process_combination(feature_selector_name, ml_model_name):
            all_selected_features = []
            all_importances = []

            # Loop through the training/test splits generated by LOOCV
            for train_index, test_index in loo.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # Perform feature selection based on the specified selector
                if feature_selector_name == 'LASSO':
                    feature_selector = LassoCV(cv=loo, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30)).fit(X_train, y_train)
                else:
                    feature_selector = ElasticNetCV(cv=loo, max_iter=20000, tol=1e-4, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7).fit(X_train, y_train)

                # Select features with non-zero coefficients
                selected_features = X_train.columns[feature_selector.coef_ != 0]
                selected_features = selected_features[:min(len(selected_features), int(0.8 * len(y_train)))]

                # If no features are selected, skip this iteration
                if len(selected_features) == 0:
                    continue

                # Select the relevant features from the training set
                X_train_selected = X_train[selected_features]

                # Perform logistic regression for ranking the selected features based on importance
                logistic = LogisticRegression(max_iter=10000, random_state=42, solver='liblinear').fit(X_train_selected, y_train)
                importances = abs(logistic.coef_[0])
                ranked_features = sorted(zip(selected_features, importances), key=lambda x: x[1], reverse=True)

                # Store selected features and their importance
                all_selected_features.extend([f[0] for f in ranked_features])
                all_importances.extend([f[1] for f in ranked_features])

            # If no features were selected across all folds, return None
            if not all_selected_features:
                return None

            # Aggregate selected features across all folds
            unique_features = list(set(all_selected_features))
            frequency = Counter(all_selected_features)
            avg_importance = {feature: np.mean([imp for feat, imp in zip(all_selected_features, all_importances) if feat == feature])
                              for feature in unique_features}

            # Calculate a composite score for each feature based on frequency and importance
            composite_scores = {feature: 0.5 * (frequency[feature] / loo.get_n_splits(X)) + 0.5 * (avg_importance[feature] / sum(avg_importance.values()))
                                for feature in unique_features}
            sorted_features = sorted(composite_scores.items(), key=lambda x: x[1], reverse=True)

            # Evaluate different feature subsets with varying thresholds
            best_overall_performance = 0
            best_overall_setup = {}
            best_percentage = 0
            thresholds = np.linspace(0.05, 0.95, 19)

            for i in range(1, min(len(sorted_features), int(0.8 * len(y))) + 1):
                selected_features = [feature[0] for feature in sorted_features[:i]]
                best_performance_for_features = 0
                best_threshold_for_features = None

                for threshold in thresholds:
                    fold_f1_scores = []

                    # Perform LOOCV prediction with the selected features and model
                    for train_index, test_index in loo.split(X):
                        X_train, X_test = X.iloc[train_index][selected_features], X.iloc[test_index][selected_features]
                        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                        # Use the appropriate model for prediction
                        if ml_model_name == 'XGBoost':
                            model = xgb.XGBClassifier(n_estimators=300, use_label_encoder=False, eval_metric='logloss', random_state=42)
                        elif ml_model_name == 'RandomForest':
                            model = RandomForestClassifier(n_estimators=300, random_state=42)
                        else:
                            model = ElasticNetCV(cv=5, max_iter=10000, alphas=np.logspace(-6, -2, 30), l1_ratio=0.7)

                        # Fit the model, make predictions and compute F1 score for the current fold
                        model.fit(X_train, y_train)
                        test_prediction = (model.predict(X_test) >= threshold).astype(int) if ml_model_name == 'ElasticNet' else (model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
                        f1_score_current = f1_score(y_test, test_prediction, average='macro')
                        fold_f1_scores.append(f1_score_current)

                    # Calculate the average F1 score for the current threshold
                    f1_score_avg = np.mean(fold_f1_scores)

                    # Record the best performance and threshold for the features
                    if f1_score_avg > best_performance_for_features:
                        best_performance_for_features = f1_score_avg
                        best_threshold_for_features = threshold

                # Update the best overall performance if it improves
                if best_performance_for_features > best_overall_performance:
                    best_overall_performance = best_performance_for_features
                    best_overall_setup = {
                        'features': selected_features,
                        'threshold': best_threshold_for_features,
                        'performance': best_performance_for_features,
                        'feature_selection_method': feature_selector_name,
                        'ml_model': ml_model_name
                    }

            return {
                'feature_selection_method': best_overall_setup['feature_selection_method'],
                'ml_model': best_overall_setup['ml_model'],
                'best_features': best_overall_setup['features'],
                'features_length': len(best_overall_setup['features']),
                'best_threshold': best_overall_setup['threshold'],
                'best_performance': best_overall_setup['performance']
            }

        # Process combinations of feature selection and prediction models
        combinations = [('LASSO', 'XGBoost'), ('ElasticNet', 'XGBoost')]
        results_per_combination = [process_combination(fs, ml) for fs, ml in combinations]
        best_combination = max(results_per_combination, key=lambda x: x['best_performance'] if x is not None else 0)

        return time_point, best_combination

    # Process relative time points within the specified range
    time_points = np.sort(data['Relative_timepoint'].unique())
    results_parallel = Parallel(n_jobs=-1)(delayed(process_time_point)(tp) for tp in time_points)
    results = {tp: result for tp, result in results_parallel if result is not None}

    return results

# Evaluate the model
results_relative = evaluate_models(filtered_data)

# Print results
print("Results for Celiac Disease Prediction by Relative Time Point:")
for time_point, res in results_relative.items():
    if res is not None:
        print(f"Relative Time Point: {time_point}")
        print(f"  Feature Selection Method: {res['feature_selection_method']}")
        print(f"  Machine Learning Model: {res['ml_model']}")
        print(f"  Best F1 Score: {res['best_performance']}")
        print(f"  Best Threshold: {res['best_threshold']}")
        print(f"  Features Used: {res['best_features']}")
        print(f"  Features Length: {res['features_length']}")
        print("-" * 40)

In [ ]:
# Create a line chart to show the F1 score for each time point
# Sorting the time points
sorted_features_time_points = [12, 15]
sorted_features_f1_scores = [79, 83]

# Create the line chart
plt.figure(figsize=(8, 4))

# Plot the line with points and set line color, marker size, and style
plt.plot(sorted_features_time_points, sorted_features_f1_scores, marker='o', linestyle='-', color='blue', markersize=10)

# Add labels to each point
for i, score in enumerate(sorted_features_f1_scores):
    plt.text(sorted_features_time_points[i], score + 1, f"{score}", ha='center', va='bottom', fontsize=14)

# Add titles and labels
plt.title('F1 Score by Age for CeD Prediction for All Subjects using Combined Abundance Data', fontsize=14, color='blue', pad=20)
plt.xlabel('Age (Months)', fontsize=14)
plt.ylabel('Average F1 Score', fontsize=14)

# Adjust the axis limits
plt.xticks(sorted_features_time_points, fontsize=12)
plt.ylim(0, 100)
plt.yticks(fontsize=12)

# Show the plot with tight layout
plt.tight_layout()
plt.show()

In [ ]:
# Create a line chart to show the F1 score for each time point
# Sorting the time points
sorted_features_time_points = [12, 15]
sorted_features_f1_scores = [100, 88]

# Create the line chart
plt.figure(figsize=(8, 4))

# Plot the line with points and set line color, marker size, and style
plt.plot(sorted_features_time_points, sorted_features_f1_scores, marker='o', linestyle='-', color='blue', markersize=10)

# Add labels to each point
for i, score in enumerate(sorted_features_f1_scores):
    plt.text(sorted_features_time_points[i], score + 1, f"{score}", ha='center', va='bottom', fontsize=14)

# Add titles and labels
plt.title('F1 Score by Age for Early vs. Late Onset Prediction using Combined Abundance Data', fontsize=14, color='blue', pad=20)
plt.xlabel('Age (Months)', fontsize=14)
plt.ylabel('Average F1 Score', fontsize=14)

# Adjust the axis limits
plt.xticks(sorted_features_time_points, fontsize=12)
plt.ylim(0, 100)
plt.yticks(fontsize=12)

# Show the plot with tight layout
plt.tight_layout()
plt.show()

In [ ]:
# Create a line chart to show the F1 score for each time point
# Sorting the time points
sorted_features_time_points = [-18, -15, -12, -9, -6, -3, 0]
sorted_features_f1_scores = [87, 100, 85, 91, 91, 82, 94]

# Create the line chart
plt.figure(figsize=(8, 4))

# Plot the line with points and set line color, marker size, and style
plt.plot(sorted_features_time_points, sorted_features_f1_scores, marker='o', linestyle='-', color='blue', markersize=10)

# Add labels to each point
for i, score in enumerate(sorted_features_f1_scores):
    plt.text(sorted_features_time_points[i], score + 1, f"{score}", ha='center', va='bottom', fontsize=14)

# Add titles and labels
plt.title('F1 Score by Relative Time Point for CeD Prediction using Combined Abundance Data', fontsize=14, color='blue', pad=20)
plt.xlabel('Age (Months)', fontsize=14)
plt.ylabel('Average F1 Score', fontsize=14)

# Adjust the axis limits
plt.xticks(sorted_features_time_points, fontsize=12)
plt.ylim(0, 100)
plt.yticks(fontsize=12)

# Show the plot with tight layout
plt.tight_layout()
plt.show()